# Tangle surface

A difficult example is the so-called "tangle surface" (see, e.g., [here](http://paulbourke.net/geometry/tangle/)), given by

$$(x^4 + y^4 + z^4  - 5(x^2 + y^2 + z^2) + 11.8 = 0.$$

This turns out to be quite difficult to calculate with interval methods, as reflected by the increased calculation times.

In [1]:
using ValidatedNumerics, IntervalConstraintProgramming

In [2]:
tangle = @constraint (x^4 + y^4) + z^4  - 5*((x^2 + y^2) + z^2) + 11.8 == 0.0

X = IntervalBox(-5..5, -5..5, -5..5)

@time paving = pave(tangle, X, 1);


forward = IntervalConstraintProgramming.GeneratedFunction(Symbol[:x,:y,:z],Symbol[:_z_1_,:_z_2_,:_z_3_,:_z_4_,:_z_5_,:_z_6_,:_z_7_,:_z_8_,:_z_9_,:_z_10_,:_z_11_,:_z_12_,:_z_13_],quote 
    _z_1_ = x ^ 4
    _z_2_ = y ^ 4
    _z_3_ = _z_1_ + _z_2_
    _z_4_ = z ^ 4
    _z_5_ = _z_3_ + _z_4_
    _z_6_ = x ^ 2
    _z_7_ = y ^ 2
    _z_8_ = _z_6_ + _z_7_
    _z_9_ = z ^ 2
    _z_10_ = _z_8_ + _z_9_
    _z_11_ = 5_z_10_
    _z_12_ = _z_5_ - _z_11_
    _z_13_ = _z_12_ + 11.8
end)
backward = IntervalConstraintProgramming.GeneratedFunction(Symbol[:x,:y,:z,:_z_1_,:_z_2_,:_z_3_,:_z_4_,:_z_5_,:_z_6_,:_z_7_,:_z_8_,:_z_9_,:_z_10_,:_z_11_,:_z_12_,:_z_13_],Symbol[:x,:y,:z],quote 
    (_z_13_,_z_12_,_) = plus_rev(_z_13_,_z_12_,11.8)
    (_z_12_,_z_5_,_z_11_) = minus_rev(_z_12_,_z_5_,_z_11_)
    (_z_11_,_,_z_10_) = mul_rev(_z_11_,5,_z_10_)
    (_z_10_,_z_8_,_z_9_) = plus_rev(_z_10_,_z_8_,_z_9_)
    (_z_9_,z,_) = power_rev(_z_9_,z,2)
    (_z_8_,_z_6_,_z_7_) = plus_rev(_z_8_,_z_6_,_z_7_)
    (_z_7_,y,_) 

In [3]:
@time refine!(paving, 0.5)

inner = paving.inner
boundary = paving.boundary;

 41.695163 seconds (87.37 M allocations: 3.235 GB, 3.45% gc time)


In [24]:
length(paving.boundary)

9240

In [4]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors

In [5]:
positions = Point{3, Float32}[(mid(x)) for x in vcat(inner, boundary)]
scales = Vec3f0[Vec3f0([diam(x) for x in xx]) for xx in vcat(inner, boundary)]

zs = Float32[x[3] for x in positions]
minz = minimum(zs)
maxz = maximum(zs)

xs = Float32[x[1] for x in positions]
minx = minimum(xs)
maxx = maximum(xs)

colors = RGBA{Float32}[RGBA( (zs[i]-minz)/(maxz-minz), (xs[i]-minx)/(maxx-minx), 0f0, 0.3f0)
        for i in 1:length(inner)+length(boundary)]
#colors2 = RGBA{Float32}[RGBA( 0.5f0, 0.5f0, 0.5f0, 0.02f0) for x in boundary];
#colors = vcat(colors1, colors2);

In [6]:
cube_particle = HyperRectangle{3, Float32}(Vec3f0(-0.5, -0.5, -0.5), Vec3f0(1, 1, 1))

window = glscreen()

vis = visualize( (cube_particle, positions), scale=scales, color=colors)

_view(vis)


@async renderloop(window)

Task (runnable) @0x000000010fe61d50